In [ ]:
import numpy as np
import plotly.graph_objects as go

# Parameters for the spinning black holes
mass = 1.0
spin = 1.4
grid_size = 25
speed_of_light = 299792458

# Orbital motion parameters
orbital_radius = 0.75  # Radius of the orbital motion
orbital_period = 5.0  # Orbital period in seconds
angular_velocity = 3 * np.pi / orbital_period  # Angular velocity

x = np.linspace(-4, 4, grid_size)
y = np.linspace(-4, 4, grid_size)
z = np.linspace(-4, 4, grid_size)
X, Y, Z = np.meshgrid(x, y, z)

fig = go.Figure()

num_frames = 100
t_values = np.linspace(0, 2 * np.pi, num_frames)
delay_factor = 0.5

frames = []  # Define the frames list

# Function to calculate the gravitational wave vector
def calculate_gravitational_wave_vector(x, y, z, t, position1, position2):
    r1 = np.sqrt((x - position1[0])**2 + (y - position1[1])**2 + z**2)
    r2 = np.sqrt((x - position2[0])**2 + (y - position2[1])**2 + z**2)
    theta1 = np.arctan2(y - position1[1], x - position1[0])
    theta2 = np.arctan2(y - position2[1], x - position2[0])
    phi1 = np.arctan2(z, np.sqrt((x - position1[0])**2 + (y - position1[1])**2))
    phi2 = np.arctan2(z, np.sqrt((x - position2[0])**2 + (y - position2[1])**2))

    # Orbital motion equations
    position1[0] = orbital_radius * np.cos(angular_velocity * t)
    position1[1] = orbital_radius * np.sin(angular_velocity * t)
    position2[0] = -orbital_radius * np.cos(angular_velocity * t)
    position2[1] = -orbital_radius * np.sin(angular_velocity * t)

    amplitude1 = np.sin(theta1 + spin * r1 - t)
    amplitude2 = np.sin(theta2 + spin * r2 - t)
    direction_x = (-np.sin(phi1) * np.cos(theta1) * amplitude1 +
                    -np.sin(phi2) * np.cos(theta2) * amplitude2)
    direction_y = (-np.sin(phi1) * np.sin(theta1) * amplitude1 +
                    -np.sin(phi2) * np.sin(theta2) * amplitude2)
    direction_z = (np.cos(phi1) * amplitude1 + np.cos(phi2) * amplitude2)

    return direction_x, direction_y, direction_z

# Create a 3D scatter plot for the gravitational wave vectors
fig = go.Figure(data=go.Cone(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    u=np.zeros_like(X.flatten()),
    v=np.zeros_like(Y.flatten()),
    w=np.zeros_like(Z.flatten()),
    sizemode="absolute",
    sizeref=0.1,  # Adjust cone size for visibility
    anchor="tail",
    colorscale='Viridis',  # Use a uniform color scale
    showscale=False,  # Hide the color scale
))

# Initialize positions for orbital motion
position1 = np.array([orbital_radius, 0.0, 0.0])
position2 = np.array([-orbital_radius, 0.0, 0.0])

# Define initial positions for black hole markers and axis
initial_position1 = position1.copy()
initial_position2 = position2.copy()

# Create markers for both black holes
black_hole_markers = go.Scatter3d(x=[initial_position1[0], initial_position2[0]],
                                   y=[initial_position1[1], initial_position2[1]],
                                   z=[initial_position1[2], initial_position2[2]],
                                   mode='markers',
                                   marker=dict(color='black', size=5, opacity=1))

# Create lines for the axis of rotation
axis_lines = [go.Scatter3d(x=[initial_position1[0], initial_position1[0]],
                           y=[initial_position1[1], initial_position1[1]],
                           z=[initial_position1[2] - 2.5, initial_position1[2] + 2.5],
                           mode='lines', line=dict(color='red', width=1)),
              go.Scatter3d(x=[initial_position2[0], initial_position2[0]],
                           y=[initial_position2[1], initial_position2[1]],
                           z=[initial_position2[2] - 2.5, initial_position2[2] + 2.5],
                           mode='lines', line=dict(color='blue', width=1))]

# Add the markers and axis lines to the figure
fig.add_trace(black_hole_markers)
fig.add_traces(axis_lines)

# Set plot title and axis labels
fig.update_layout(title='Gravitational Wave Vectors and Axis of Rotation around Two Spinning Black Holes with Parallel Axes',
                  scene=dict(
                      xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
                      aspectratio=dict(x=1, y=1, z=1),
                  ))

for i, t in enumerate(t_values):
    U, V, W = calculate_gravitational_wave_vector(X, Y, Z, t - (delay_factor * i / num_frames) * 2 * np.pi * mass / speed_of_light, position1, position2)

    # Calculate opacity based on propagation delay
    opacity = (i / num_frames) * 0.8 + 0.2

    frame_data = go.Cone(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        u=U.flatten(),
        v=V.flatten(),
        w=W.flatten(),
        sizemode="absolute",
        sizeref=0.95,  # Adjust cone size for visibility
        anchor="tail",
        colorscale='edge',  # Use a uniform color scale
        showscale=True,  # Hide the color scale
        opacity=opacity,  # Apply opacity based on propagation delay
    )

    # Update the positions of black hole markers and axis
    position1[0] = orbital_radius * np.cos(angular_velocity * t)
    position1[1] = orbital_radius * np.sin(angular_velocity * t)
    position2[0] = -orbital_radius * np.cos(angular_velocity * t)
    position2[1] = -orbital_radius * np.sin(angular_velocity * t)
    black_hole_markers.x = [position1[0], position2[0]]
    black_hole_markers.y = [position1[1], position2[1]]
    black_hole_markers.z = [position1[2], position2[2]]
    axis_lines[0].x = [position1[0], position1[0]]
    axis_lines[0].y = [position1[1], position1[1]]
    axis_lines[0].z = [position1[2] - 2.5, position1[2] + 2.5]
    axis_lines[1].x = [position2[0], position2[0]]
    axis_lines[1].y = [position2[1], position2[1]]
    axis_lines[1].z = [position2[2] - 2.5, position2[2] + 2.5]

    frame = go.Frame(data=[frame_data, black_hole_markers] + axis_lines)
    frames.append(frame)

fig.frames = frames

fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(
                    label='Play',
                    method='animate',
                    args=[
                        None,
                        {
                            'frame': {'duration': 50, 'redraw': True},
                            'fromcurrent': True,
                            'transition': {'duration': 0}
                        }
                    ]
                ),
                dict(
                    label='Pause',
                    method='animate',
                    args=[
                        [None],
                        {
                            'frame': {'duration': 0, 'redraw': False},
                            'mode': 'immediate',
                            'transition': {'duration': 0}
                        }
                    ]
                )
            ],
            showactive=False,
            x=0.1,
            y=0,
            xanchor='right',
            yanchor='top'
        )
    ]
)

# Display the plot
fig.show()
